In [62]:
import pandas; import os; import pickle

# Pre-Questionnaire Read

Read in the data from pre-experiment questionnaire and check age range and participant genders

In [80]:
## compile full gaze df's from each subject
paths   = ['../sustained_attention_experiment/data/', '../variable_attention_experiment/data/']
groups  = ['group2']
list_a  = []
list_b  = []

for exp,l in zip(paths,[list_a,list_b]):
    for group in groups:
        print(exp); print(group); print()
        
        subjects = os.listdir(exp+group)
        #print(subjects)
        
        for s in subjects:
            
            if s != '.DS_Store':
                
                print(s)
                
                k = [i for i in os.listdir(exp+group+'/'+s) if 'pre_q' in i] 
                
                if len(k)>0:
                    
                    res = exp+group+'/'+s+'/'+k[0]
                    
                    with open(res,'rb') as fp:
                        data = pickle.load(fp)
                    
                    l.append(data)
                    
                else:
                    print(s)
                    
                
#                 print('df_pres_gaze_data.csv' in os.listdir(exp+group+'/'+s))
#                 if 'df_pres_gaze_data.csv' in os.listdir(exp+group+'/'+s):
#                     print(s)
#                     gaze.append(pd.read_csv(exp+group+'/'+s+'/df_pres_gaze_data.csv'))

# gaze_df = pd.concat(gaze)
# gaze_df.to_csv('../parsed_data/full_pres_gaze.csv') 




# os.listdir('../variable_attention_experiment/data/group1/8_2019_Feb_28')

../sustained_attention_experiment/data/
group2

34_2019_Oct_13
40_2019_Oct_17
7_2019_Oct_13
13_2019_Oct_14
39_2019_Oct_17
16_2019_Oct_08
2_2019_Oct_06
8_2019_Oct_13
32_2019_Oct_15
15_2019_Oct_14
5_2019_Oct_17
1_2019_Oct_13
10_2019_Oct_14
19_2019_Oct_15
4_2019_Oct_13
28_2019_Oct_09
7_2019_Oct_13_b
14_2019_Oct_07
3_2019_Oct_13
18_2019_Oct_08
38_2019_Oct_17
35_2019_Oct_13
11_2019_Oct_14_b
20_2019_Oct_08
36_2019_Oct_13
5_2019_Oct_13
11_2019_Oct_14
0_2019_Oct_13
25_2019_Oct_08
9_2019_Oct_14
../variable_attention_experiment/data/
group2

56_2020_Feb_21
17_2019_Nov_18
12_2019_Nov_17
20_2019_Nov_19
30_2020_Jan_13
25_2020_Jan_24
11_2019_Nov_17
18_2019_Nov_19
9_2019_Nov_16
29_2020_Jan_13
14_2019_Nov_17
21_2019_Nov_19
27_2020_Jan_15
6_2019_Nov_15
16_2019_Nov_18
26_2020_Jan_16
28_2020_Jan_13
8_2019_Nov_16
15_2019_Nov_18
5_2019_Nov_15
19_2019_Nov_19
10_2019_Nov_16
0_2020_Feb_07


In [81]:
sust_df = pd.DataFrame(list_a)
sust_df.groupby([1], as_index=False).count()
# sust_df

,1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,--,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,Female,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
2,Male,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12


In [82]:
var_df  = pd.DataFrame(list_b)
var_df.groupby([1], as_index=False).count()

,1,0,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Female,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
1,Male,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [76]:
print(str(sust_df[0].min()))
print(str(sust_df[0].max()))
print()
print(str(var_df[0].min()) )
print(str(var_df[0].max()) )

18
20

18
21 


In [72]:
list_b

[[21 ',
  'Female',
  'No',
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  'some college',
  'No',
  'Yes',
  'No',
  '',
  5',
  0',
  'A little slugglish'],
 [18',
  'Female',
  'No',
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  'some college',
  'No',
  'Yes',
  'No',
  '',
  8.5',
  0',
  'A little slugglish'],
 [21',
  'Male',
  'Yes',
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  'some college',
  'No',
  'No',
  'No',
  '',
  6',
  2',
  'Very alert'],
 [19',
  'Female',
  'No',
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  'some college',
  'No',
  'Yes',
  'No',
  '',
  9',
  0',
  'A little alert'],
 [19',
  'Female',
  'No',
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  'some college',
  'No',
  'Yes',
  'No',
  '',
  5',
  0',
  'Neutral'],
 [19',
  'Female',
  'No',
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  'high school graduate',
  'No',
  'Yes',
  'No',
  '',
 